# Simulation Project OLSGS 
## Group 7

### Part 1: DES simulation of a tandem line 

#### 1. States: 
States are given by $$(x_1, x_2, x_3, x_4, \delta_1, \delta_2, \delta_3)$$ with $x_i$ number of jobs at station $i$ including the server and $\delta_i$ indicating whether station $i$ is available $(\delta_i=1)$ or blocked $(\delta_i=0)$. 

#### 2. Events:
$\alpha =$ arrival of a new job, $\beta_1 =$ service completion of machine 1, $\beta_2 =$ service completion of machine 2, $\beta_3 =$ service completion of machine 3, $\beta_4 =$ service completion of machine 4. 

#### 3. Event lists: 
$L(0,)


In [73]:
import numpy as np
import numpy.random as rnd
import scipy.stats as st
import matplotlib.pyplot as plt
import scipy as sc
import statsmodels.api as sm

In [80]:
# Instance 
# Number of service stations, each station with a single service place Si = 1, for i = 1, ..., 4 
I = 4
S1 = 1
S2 = 2
S3 = 3
S4 = 1

# Buffer places per station 
C1 = 5
C2 = 6
C3 = 6
C4 = 3

# Rates 
lambda_rate = 2.1
mu_1 = 2.9
mu_2 = 2.2
mu_3 = 2.2
mu_4 = 2.8 




In [75]:
class State(object): 
    def __init__(self, _x1 = 0, _x2 = 0, _x3=0, _x4=0, _delta1=1, _delta2=1, _delta3=1):
        self.x1 = _x1
        self.x2 = _x2 
        self.x3 = _x3
        self.x4 = _x4
        self.delta1 = _delta1
        self.delta2 = _delta2
        self.delta3 = _delta3

    def printState(self):
        print('(', self.x1, ', ', self.x2, ', ', self.x3, ', ', self.x4, ', ', self.delta1, ', ', self.delta2, ', ', self.delta3, ')')

class Event(object): 
    def __init__(self, _time = np.inf, _type = ''):
        self.time = _time
        self.type = _type 
    
    def getTime(self): 
        return self.time
    
    def getType(self): 
        return self.type
    
    def printEvent(self): 
        print('(', self.time, ',', self.type, ')', end = ' ')
        

class EventList(object):
    def __init__(self, _elist=[]):
        self.elist = _elist 
    
    def addEvent(self, event): 
        if len(self.elist)==0: 
            self.elist = [event]
        else:
            time_event = event.getTime()
            if time_event > self.elist[-1].getTime():
                self.elist.append(event)
            else: 
                event_start = next(event for event in self.elist if event.getTime() > time_event)
                event_index = self.elist.index(event_start)
                self.elist.insert(event_index, event)
    
    def getFirstEvent(self):
        ''' 
        Gets first item on the list and removes item from the list 
        '''
        first_event = self.elist.pop(0)
        return first_event
    
    def printEventList(self): 
        for event in self.elist:
            event.printEvent()
        print('')
    

class CounterSimulation(object):
    def __init__(self, _numServed = 0): 
        self.numServed = _numServed 

    def printNumberServed(self):
        print(self.numServed)


In [76]:
def randomExponential(rate): 
    return -np.log(rnd.random()) / rate 
    

## Events

In [91]:
def processArrivalEvent(state, eventTime, eventList): 
    # Check if queue is full 
    if state.x1 == C1 + S1: 
        
        # Schedule new arrival
        interArrivalTime = randomExponential(lambda_rate)
        arrivalEvent = Event(eventTime + interArrivalTime, 'arrivalEvent')
        eventList.addEvent(arrivalEvent)

    else: 

        # Update state 
        state.x1 += 1
    
        # Check if server is blocked 
        if state.delta1 == 1: 
            
            # Check if there is a queue 
            if state.x1 <= S1:
                
                # Schedule departure 
                serviceTime = randomExponential(mu_1)
                departureEvent = Event(eventTime + serviceTime, 'departureEvent1')
                eventList.addEvent(departureEvent)
         
        # Schedule next arrival: interarrival time ~exp(lambda)
        interArrivalTime = randomExponential(lambda_rate)
        arrivalEvent = Event(eventTime + interArrivalTime, 'arrivalEvent')
        eventList.addEvent(arrivalEvent)
    
def departureEventServer1(state, eventTime, eventList):
    # Check if next server is available: 
    if state.x2 == C2 + S2: 
        state.delta1 = 0 
    
    else: 
        # Update state
        state.x1 -= 1

        # Add customer to new server 
        state.x2 += 1

        # Check if there is a queue at server 2
        if state.x2 <= S2: # No queue

            # Schedule departure from server 2 
            serviceTime = randomExponential(mu_2)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent2')
            eventList.addEvent(departureEvent)
        
        ## Processing next customer: 
        # Check if there is a queue 
        if state.x1 >= S1:

            # Schedule departure 
            serviceTime = randomExponential(mu_1)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent1')
            eventList.addEvent(departureEvent)
        
    
def departureEventServer2(state, eventTime, eventList):
    # Check if next server is available
    if state.x3 == C3 + S3: 
        state.delta2 = 0 
    
    else: 
        # Update state
        state.x2 -= 1

        # Add customer to new server 
        state.x3 += 1

        # Check if there is a queue at server 3
        if state.x3 <= S3: # No queue, direct service 
            
            # Schedule departure from server 3 
            serviceTime = randomExponential(mu_3)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent3')
            eventList.addEvent(departureEvent)

        # Activate server 1 if necessary 
        if state.delta1 == 0:
            state.delta1 = 1
            
            # Schedule departure from server 1 
            if state.x1 >= 1:  
                serviceTime = randomExponential(mu_1)
                departureEvent = Event(eventTime + serviceTime, 'departureEvent1')
                eventList.addEvent(departureEvent)
        
        ## Processing next customer: 
        # Check if there is a queue 
        if state.x2 >= S2: 

            # Schedule departure 
            serviceTime = randomExponential(mu_2)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent2')
            eventList.addEvent(departureEvent)


def departureEventServer3(state, eventTime, eventList):
    # Check if next server is available
    if state.x4 == C4 + S4: 
        state.delta3 = 0 
    
    else: 
        # Update state
        state.x3 -= 1

        # Add customer to new server 
        state.x4 += 1

        # Check if there is a queue at server 4
        if state.x4 <= S4: # No queue, direct service 
            
            # Schedule departure from server 4
            serviceTime = randomExponential(mu_4)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent4')
            eventList.addEvent(departureEvent)
        
        # Activate server 2 if necessary 
        if state.delta2 == 0:
            state.delta2 = 1
            
            # Schedule departure from server 2
            if state.x2 >= S2:  
                serviceTime = randomExponential(mu_2)
                departureEvent = Event(eventTime + serviceTime, 'departureEvent2')
                eventList.addEvent(departureEvent)
        
        # Activate server 1 if necessary 
        if state.delta1 == 0: 
            state.delta1 = 1

            # Schedule departure from server 1
            if state.x1 >= S1: 
                serviceTime = randomExponential(mu_1)
                departureEvent = Event(eventTime + serviceTime, 'departureEvent1')
                eventList.addEvent(departureEvent)

        ## Processing next customer: 
        # Check if there is a queue 
        if state.x3 >= S3: 

            # Schedule departure 
            serviceTime = randomExponential(mu_3)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent3')
            eventList.addEvent(departureEvent)

def departureEventServer4(state, eventTime, eventList, counter):
    # Update state
    state.x4 -= 1

    # Update counters
    counter.numServed += 1 


    # Activate server 3 if necessary 
    if state.delta3 == 0:
        state.delta3 = 1
        
        # Schedule departure from server 3 
        if state.x3 >= S3:  
            serviceTime = randomExponential(mu_3)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent3')
            eventList.addEvent(departureEvent)
    
    # Activate server 2 if necessary 
    if state.delta2 == 0: 
        state.delta2 = 1

        if state.x2 >= S2: 
            serviceTime = randomExponential(mu_2)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent2')
            eventList.addEvent(departureEvent)

    # Activate server 1 if necessary 
    if state.delta1 == 0:
        state.delta1 = 1

        if state.x1 >= S1: 
            serviceTime = randomExponential(mu_1)
            departureEvent = Event(eventTime + serviceTime, 'departureEvent1')
            eventList.addEvent(departureEvent)
        
    ## Processing next customer: 
    # Check if there is a queue 
    if state.x4 >= S4: 

        # Schedule departure 
        serviceTime = randomExponential(mu_4)
        departureEvent = Event(eventTime + serviceTime, 'departureEvent4')
        eventList.addEvent(departureEvent)

Simulation 

In [92]:
def simulationRun(TimeHorizon):
    currentTime = 0
    counter = CounterSimulation()
    state = State()
    eventList = EventList()

    # initialisation of the simulation 
    interArrivalTime = randomExponential(lambda_rate)
    arrivalEvent = Event(currentTime + interArrivalTime, 'arrivalEvent')
    eventList.addEvent(arrivalEvent)

    while currentTime < TimeHorizon:
        event = eventList.getFirstEvent()
        timeEvent = event.getTime()
        typeEvent = event.getType()

        if typeEvent == 'arrivalEvent': 
            processArrivalEvent(state, timeEvent, eventList)
        elif typeEvent == 'departureEvent1': 
            departureEventServer1(state, timeEvent, eventList)
        elif typeEvent == 'departureEvent2': 
            departureEventServer2(state, timeEvent, eventList)
        elif typeEvent == 'departureEvent3': 
            departureEventServer3(state, timeEvent, eventList)
        else: 
            departureEventServer4(state, timeEvent, eventList, counter)
        
        state.printState()
        currentTime = timeEvent
    
    state.printState()

    return counter.printNumberServed()
        
    
    

In [93]:
def main1():
    simulationRun(100)

main1()


( 1 ,  0 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 0 ,  1 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 0 ,  0 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 1 ,  0 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 0 ,  1 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 1 ,  1 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 0 ,  2 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 0 ,  1 ,  2 ,  0 ,  1 ,  1 ,  1 )
( 1 ,  1 ,  2 ,  0 ,  1 ,  1 ,  1 )
( 1 ,  1 ,  1 ,  1 ,  1 ,  1 ,  1 )
( 1 ,  0 ,  2 ,  1 ,  1 ,  1 ,  1 )
( 1 ,  0 ,  2 ,  0 ,  1 ,  1 ,  1 )
( 1 ,  0 ,  1 ,  1 ,  1 ,  1 ,  1 )
( 1 ,  0 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 2 ,  0 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 2 ,  0 ,  0 ,  1 ,  1 ,  1 ,  1 )
( 2 ,  0 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 3 ,  0 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 2 ,  1 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 3 ,  1 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 4 ,  1 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 3 ,  2 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 4 ,  2 ,  0 ,  0 ,  1 ,  1 ,  1 )
( 4 ,  1 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 3 ,  2 ,  1 ,  0 ,  1 ,  1 ,  1 )
( 3 ,  1 ,  2 ,  0 ,  1 ,  1 ,  1 )
( 2 ,  2 ,  2 ,  0 ,  1 ,  1 ,  1 )
( 3 ,  2 ,  2 ,  0 ,  1 ,  1